## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]


In [4]:
# Create a list of at least 10 column names to use as X data
# Create X_df using your selected columns
X_df = attrition_df[['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction',\
                     'OverTime', 'StockOptionLevel', 'WorkLifeBalance', \
                    'YearsAtCompany', 'YearsSinceLastPromotion', 'NumCompaniesWorked']]

# Show the data types for X_df
print(X_df.dtypes)


Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object


In [5]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=4)


In [6]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train['OverTime'] = X_train['OverTime'].map({'No': 0, 'Yes': 1})
X_test['OverTime'] = X_test['OverTime'].map({'No': 0, 'Yes': 1})

overtime_counts_train = X_train['OverTime'].value_counts()
overtime_counts_train.index = overtime_counts_train.index.map({0: 'No', 1: 'Yes'})

# Display the result
print(overtime_counts_train)


OverTime
No     787
Yes    315
Name: count, dtype: int64


In [7]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [8]:
# Create a OneHotEncoder for the Department column
ohe_department = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
ohe_department.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_department = ohe_department.transform(y_train[['Department']])
y_test_department = ohe_department.transform(y_test[['Department']])

y_train_department


array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [9]:
# Create a OneHotEncoder for the Attrition column
ohe_attrition = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
ohe_attrition.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition = ohe_attrition.transform(y_train[['Attrition']])
y_test_attrition = ohe_attrition.transform(y_test[['Attrition']])

y_train_attrition

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]])

## Create, Compile, and Train the Model

In [10]:
# Find the number of columns in the X training data
columns = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(columns,))

# Create at least two shared layers
shared_layer_1 = layers.Dense(64, activation='relu', name='shared1')(input_layer)
shared_layer_2 = layers.Dense(128, activation='relu', name='shared2')(shared_layer_1)

In [11]:
# Create a branch for Department
# with a hidden layer and an output layer
# Create the hidden layer
dept_hidden_layer = layers.Dense(32, activation='relu', name='department_hidden')(shared_layer_2)

# Create the output layer
dept_output_layer = layers.Dense(y_train_department.shape[1], activation='softmax', name='dept_output')\
                    (dept_hidden_layer)


In [12]:
# Create a branch for Attrition
# with a hidden layer and an output layer
# Create the hidden layer
attrition_hidden_layer = layers.Dense(32, activation='relu', name='attrition_hidden')(shared_layer_2)

# Create the output layer
attrition_output_layer = layers.Dense(y_train_attrition.shape[1], activation='sigmoid',name='attrition_output')\
                        (attrition_hidden_layer)


In [13]:
# Create the model
model = Model(inputs=input_layer, outputs=[dept_output_layer, attrition_output_layer], name='model')

# Compile the model
model.compile(
    optimizer='adam',
    loss={
        'dept_output': 'categorical_crossentropy',
        'attrition_output': 'binary_crossentropy',
    },
    metrics={
        'dept_output': 'accuracy',
        'attrition_output': 'accuracy',
    },
)

# Summarize the model
model.summary()

Model: "model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared1 (Dense)     │ (None, 64)        │        704 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared2 (Dense)     │ (None, 128)       │      8,320 │ shared1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_hidden   │ (None, 32)        │      4,128 │ shared2[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_hidden    │ (None, 32)        │      4,128 │ shared2[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dept_output (Dense) │ (None, 3)         │         99 │ department_hidde… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         66 │ attrition_hidden… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Train the model
history = model.fit(X_train_scaled,{'dept_output': y_train_department, 'attrition_output': y_train_attrition},\
                    epochs=50, batch_size=32, verbose=1)


Epoch 1/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - attrition_output_accuracy: 0.7250 - attrition_output_loss: 0.5945 - dept_output_accuracy: 0.5968 - dept_output_loss: 0.9418 - loss: 1.5363
Epoch 2/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.8363 - attrition_output_loss: 0.4301 - dept_output_accuracy: 0.6627 - dept_output_loss: 0.7532 - loss: 1.1833 
Epoch 3/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.8414 - attrition_output_loss: 0.3891 - dept_output_accuracy: 0.6557 - dept_output_loss: 0.7592 - loss: 1.1483 
Epoch 4/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.8518 - attrition_output_loss: 0.3772 - dept_output_accuracy: 0.6798 - dept_output_loss: 0.7329 - loss: 1.1102 
Epoch 5/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.8730 - attrition_output_loss: 0.3396 - dept_output_accuracy: 0.6669 - dept_output_loss: 0.7254 - loss: 1.0649 
Epoch 6/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [15]:
# Evaluate the model with the testing data
evaluation = model.evaluate(
    X_test_scaled,
    {'dept_output': y_test_department, 'attrition_output': y_test_attrition},\
    verbose=1)

evaluation                         

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8177 - attrition_output_loss: 0.7627 - dept_output_accuracy: 0.5007 - dept_output_loss: 1.6264 - loss: 2.3934  


[2.369542360305786,
 1.5936819314956665,
 0.7478217482566833,
 0.8070651888847351,
 0.52173912525177]

In [16]:
# Print the accuracy for both department and attrition
dept_accuracy = evaluation[3]
attrition_accuracy = evaluation[4]
print(f"Department Accuracy: {dept_accuracy * 100:.2f}%")
print(f"Attrition Accuracy: {attrition_accuracy * 100:.2f}%")

Department Accuracy: 80.71%
Attrition Accuracy: 52.17%


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

**Accuracy is not always the best metric, primarily for the attrition output.**  
   Attrition is an unbalanced dataset representing far more 'No' results than 'Yes'. In this dataset, there are more than twice as many 'No' results as 'Yes'.

2. What activation functions did you choose for your output layers, and why?

**For the 'Department' output, I used softmax as it is ideal for multi-class classification with mutually exclusive data.**  
   This ensures the model predicts a single 'Department' with a high level of confidence.  
   **Sigmoid was utilized for the 'Attrition' data due to its binary classification.**  
   The binary classification involves simple 0 or 1 labels, corresponding to 'No' and 'Yes' results.

3. Can you name a few ways that this model might be improved?

**Oversampling the minority 'Yes' class or undersampling the 'No' class in attrition are techniques that could improve the model.**  
   An architectural improvement could involve the use of early stopping to prevent overfitting during training.  
   Additionally, using better evaluation metrics like AUC-ROC could provide greater insights into the attrition data.  